In [ ]:
# pip install mediapipe opencv-python
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

In [ ]:
# Setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Motion tracking for waving - MOVED OUTSIDE THE LOOP
motion_buffers = {"Left": deque(maxlen=20), "Right": deque(maxlen=20)}

def count_fingers(hand_landmarks, handedness):
    fingers = []
    tip_ids = [4, 8, 12, 16, 20]

    # Thumb (flip logic for right hand)
    if handedness == "Right":
        if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0] - 1].x:
            fingers.append(1)
        else:
            fingers.append(0)
    else:
        if hand_landmarks.landmark[tip_ids[0]].x > hand_landmarks.landmark[tip_ids[0] - 1].x:
            fingers.append(1)
        else:
            fingers.append(0)

    # Other four fingers (same logic)
    for id in range(1, 5):
        if hand_landmarks.landmark[tip_ids[id]].y < hand_landmarks.landmark[tip_ids[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers

def detect_hand_sign(fingers):
    total_fingers = sum(fingers)
    if total_fingers == 5:
        return "Open hand"
    elif fingers == [1, 0, 0, 0, 1]:
        return "Surf hand"
    elif fingers == [0, 1, 1, 0, 0]:
        return "Peace hand"
    else:
        return "Unknown"

def detect_wave(buffer, min_peaks=3, min_amplitude=0.05):
    if len(buffer) < 15:  # Need enough data points
        return False

    x_positions = np.array(buffer)
    
    # Smooth the data to reduce noise
    if len(x_positions) > 3:
        smoothed = np.convolve(x_positions, np.ones(3)/3, mode='valid')
    else:
        smoothed = x_positions
    
    if len(smoothed) < 5:
        return False
    
    # Calculate derivatives to find direction changes
    diffs = np.diff(smoothed)
    
    # Find peaks and valleys (direction changes)
    sign_changes = 0
    for i in range(1, len(diffs)):
        if (diffs[i-1] > 0 and diffs[i] < 0) or (diffs[i-1] < 0 and diffs[i] > 0):
            sign_changes += 1
    
    # Check motion amplitude
    motion_range = np.max(x_positions) - np.min(x_positions)
    
    # More lenient thresholds for better detection
    return sign_changes >= min_peaks and motion_range >= min_amplitude

# Camera loop
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip for natural view
        frame = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            hand_signs = []

            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[idx].classification[0].label

                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                fingers = count_fingers(hand_landmarks, handedness)
                sign = detect_hand_sign(fingers)

                # Track wrist x-position for waving motion
                wrist_x = hand_landmarks.landmark[0].x
                motion_buffers[handedness].append(wrist_x)

                # Replace Open Hand with Waving if waving is detected
                if sign == "Open hand" and detect_wave(motion_buffers[handedness]):
                    sign = "Waving"

                # Append full label
                hand_label = f"{handedness} {sign}"
                if hand_label not in hand_signs:
                    hand_signs.append(hand_label)

            # Combine if multiple hands
            final_sign = " | ".join(hand_signs) if hand_signs else ""

            # Show on screen
            cv2.putText(frame, final_sign, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
        else:
            # Clear motion buffers when no hands are detected
            motion_buffers["Left"].clear()
            motion_buffers["Right"].clear()

        cv2.imshow("Hand Sign Detection", frame)
        if cv2.waitKey(10) & 0xFF == 27:  # ESC to quit
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1751282694.469808 1895165 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3
W0000 00:00:1751282694.476567 1917958 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751282694.481574 1917960 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

: 